#Nhập dữ liệu

In [2]:
!pip install pyclustering


In [3]:
from pyclustering.cluster.bang import bang, bang_visualizer
from pyclustering.utils import read_sample
from pyclustering.samples.definitions import FCPS_SAMPLES

Phân cụm khách hàng theo hành vi mua hàng

In [4]:
#Tải bộ dữ liệu từ kaggle
import kagglehub

# Download latest version
path = kagglehub.dataset_download("bhanuthakurr/cleaned-contoso-dataset")

print("Path to dataset files:", path)

Path to dataset files: /root/.cache/kagglehub/datasets/bhanuthakurr/cleaned-contoso-dataset/versions/8


In [4]:
import os

# Hiển thị các tệp trong thư mục đã tải
files = os.listdir(path)
print(files)


['DimMachine.csv', 'DimScenario.csv', 'DimPromotion.csv', 'FactOnlineSales.csv', 'DimProductSubcategory.csv', 'DimProduct.csv', 'DimStore.csv', 'FactITMachine.csv', 'FactInventory.csv', 'DimCustomer.csv', 'DimAccount.csv', 'FactStrategyPlan.csv', 'DimCurrency.csv', 'DimEntity.csv', 'DimDate.csv', 'DimGeography.csv', 'FactITSLA.csv', 'DimEmployee.csv', 'DimOutage.csv', 'DimChannel.csv', 'DimProductCategory.csv', 'FactSales.csv', 'FactSalesQuota.csv', 'FactExchangeRate.csv', 'DimSalesTerritory.csv']


In [5]:
import pandas as pd

# Chọn tên file cụ thể bạn muốn hiển thị
file_name = "FactSales.csv"
file_path = os.path.join(path, file_name)

# Đọc và hiển thị nội dung của file CSV
data = pd.read_csv(file_path)
print(data.head())  # Hiển thị 5 dòng đầu tiên của bảng FactSales


   SalesKey     DateKey  channelKey  StoreKey  ProductKey  PromotionKey  \
0         1  2007-01-02           1       209         956            10   
1         2  2007-02-12           4       308         766             2   
2         3  2008-01-24           1       156        1175            11   
3         4  2008-01-13           2       306        1429            10   
4         5  2008-01-22           2       306        1133            10   

   CurrencyKey  UnitCost  UnitPrice  SalesQuantity  ReturnQuantity  \
0            1     91.05      198.0              8               0   
1            1     10.15       19.9              4               0   
2            1    209.03      410.0              9               0   
3            1    132.90      289.0              8               0   
4            1    144.52      436.2             24               0   

   ReturnAmount  DiscountQuantity  DiscountAmount  TotalCost  SalesAmount  
0           0.0                 1          39.600   

#Tiền xử lý dữ liệu

In [6]:
# Chỉ giữ lại các cột cần thiết
columns_to_keep = ['SalesQuantity', 'ReturnQuantity',
                   'ReturnAmount', 'DiscountQuantity', 'DiscountAmount',
                   'TotalCost', 'SalesAmount']

# Lọc dữ liệu, giữ lại các cột đã chỉ định
filtered_data = data[columns_to_keep]

# Hiển thị 5 dòng đầu tiên của bảng FactSales sau khi lọc
print(filtered_data.head())
filtered_data = data[columns_to_keep]

   SalesQuantity  ReturnQuantity  ReturnAmount  DiscountQuantity  \
0              8               0           0.0                 1   
1              4               0           0.0                 1   
2              9               0           0.0                 3   
3              8               0           0.0                 1   
4             24               0           0.0                 3   

   DiscountAmount  TotalCost  SalesAmount  
0          39.600     728.40     1544.400  
1           0.995      40.60       78.605  
2          61.500    1881.27     3628.500  
3          57.800    1063.20     2254.200  
4         261.720    3468.48    10207.080  


In [10]:
filtered_data.head()

,SalesQuantity,ReturnQuantity,ReturnAmount,DiscountQuantity,DiscountAmount,TotalCost,SalesAmount
0,8,0,0.0,1,39.600,728.40,1544.400
1,4,0,0.0,1,0.995,40.60,78.605
2,9,0,0.0,3,61.500,1881.27,3628.500
3,8,0,0.0,1,57.800,1063.20,2254.200
4,24,0,0.0,3,261.720,3468.48,10207.080


In [11]:
filtered_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3406089 entries, 0 to 3406088
Data columns (total 7 columns):
 #   Column            Dtype  
---  ------            -----  
 0   SalesQuantity     int64  
 1   ReturnQuantity    int64  
 2   ReturnAmount      float64
 3   DiscountQuantity  int64  
 4   DiscountAmount    float64
 5   TotalCost         float64
 6   SalesAmount       float64
dtypes: float64(4), int64(3)
memory usage: 181.9 MB


missing values

In [12]:
# Đếm số dòng chứa NA (None, NaN) của mỗi cột
missing_values = filtered_data.isnull().sum()
if missing_values.sum() == 0:
    print("Không có NaN.")
else:
    print("Có NaN, các cột có NaN là:")
    print(missing_values[missing_values > 0])

Không có NaN.


Duplicated values

In [13]:
# Loại bỏ các hàng trùng lặp
filtered_data.drop_duplicates(inplace=True)

<ipython-input-13-0689b0529068>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data.drop_duplicates(inplace=True)


In [15]:
from scipy.stats import normaltest  # Thêm import này

# Lặp qua từng cột trong filtered_data và kiểm tra phân phối chuẩn
for column in filtered_data.columns:
    stat, p_value = normaltest(filtered_data[column])
    print(f'Column {column}:')
    print(f'Statistic: {stat}, p-value: {p_value}')
    if p_value < 0.05:
        print('Không có phân phối chuẩn\n')
    else:
        print('Có phân phối chuẩn\n')

Column SalesQuantity:
Statistic: 305380.83427842887, p-value: 0.0
Không có phân phối chuẩn

Column ReturnQuantity:
Statistic: 15859.596315452078, p-value: 0.0
Không có phân phối chuẩn

Column ReturnAmount:
Statistic: 186597.17543153607, p-value: 0.0
Không có phân phối chuẩn

Column DiscountQuantity:
Statistic: 14900.914928193146, p-value: 0.0
Không có phân phối chuẩn

Column DiscountAmount:
Statistic: 198541.24495678442, p-value: 0.0
Không có phân phối chuẩn

Column TotalCost:
Statistic: 170637.76940872642, p-value: 0.0
Không có phân phối chuẩn

Column SalesAmount:
Statistic: 186702.73798115872, p-value: 0.0
Không có phân phối chuẩn



outliner

In [17]:
import numpy as np
def remove_outliers(d, filtered_data):
    for i in d:
        Q3, Q1 = np.percentile(filtered_data[i], [75 ,25])
        IQR = Q3 - Q1

        ul = Q3+1.5*IQR
        ll = Q1-1.5*IQR

        filtered_data = filtered_data[~((filtered_data[i] > ul) | (filtered_data[i] < ll))]
    return filtered_data


In [18]:

numerical = ['SalesQuantity', 'ReturnQuantity',
                   'ReturnAmount', 'DiscountQuantity', 'DiscountAmount',
                   'TotalCost', 'SalesAmount']
filtered_data = remove_outliers(numerical, filtered_data)

In [19]:
filtered_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 116403 entries, 0 to 3406069
Data columns (total 7 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   SalesQuantity     116403 non-null  int64  
 1   ReturnQuantity    116403 non-null  int64  
 2   ReturnAmount      116403 non-null  float64
 3   DiscountQuantity  116403 non-null  int64  
 4   DiscountAmount    116403 non-null  float64
 5   TotalCost         116403 non-null  float64
 6   SalesAmount       116403 non-null  float64
dtypes: float64(4), int64(3)
memory usage: 7.1 MB


#xây dựng mô hình phân lớp BANG (chưa có chạy ra)

In [20]:
# Chuyển đổi dữ liệu thành dạng mảng numpy để sử dụng trong BANG
data_array = filtered_data.values

In [21]:
# Số lượng cấp độ (levels) của BANG (có thể điều chỉnh tùy thuộc vào độ phức tạp của dữ liệu)
levels = 11

In [23]:
# Tạo một instance của thuật toán BANG
bang_instance = bang(data_array, levels)


In [24]:
# Thực hiện quá trình phân cụm
bang_instance.process()

In [25]:
# Lấy kết quả phân cụm
clusters = bang_instance.get_clusters()
noise = bang_instance.get_noise()
directory = bang_instance.get_directory()
dendrogram = bang_instance.get_dendrogram()

In [1]:
# Visualize BANG clustering results.
bang_visualizer.show_blocks(directory)
bang_visualizer.show_dendrogram(dendrogram)
bang_visualizer.show_clusters(data, clusters, noise)

NameError: name 'bang_visualizer' is not defined

In [ ]:
# Hiển thị cấu trúc phân cụm
bang_visualizer.show_blocks(directory)

In [ ]:
# Hiển thị dendrogram của quá trình phân cụm
bang_visualizer.show_dendrogram(dendrogram)

In [ ]:
# Hiển thị kết quả các cụm và noise
bang_visualizer.show_clusters(data_array, clusters, noise)